PromptTemplate

In [6]:
#字符模板
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("你是一个{name},帮我起一个具有{county}特色的{sex}名字");
prompt.format(name="算命先生", county="法国", sex="女孩");


ChatPromptTemplate

In [5]:
#对话模版具有结构，chatmodels
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个起名大师，你的名字叫{name}."),
        ("human", "你好{name}，你感觉如何？"),
        ("ai", "你好!我状态非常好！"),
        ("human", "你叫什么名字呢？"),
        ("ai", "你好，我叫{name}"),
        ("human", "{user_input}")
    ]
)

chat_template.format_messages(name="陈大师", user_input="你的爸爸是谁呢？")

[SystemMessage(content='你是一个起名大师，你的名字叫陈大师.', additional_kwargs={}),
 HumanMessage(content='你好陈大师，你感觉如何？', additional_kwargs={}, example=False),
 AIMessage(content='你好!我状态非常好！', additional_kwargs={}, example=False),
 HumanMessage(content='你叫什么名字呢？', additional_kwargs={}, example=False),
 AIMessage(content='你好，我叫陈大师', additional_kwargs={}, example=False),
 HumanMessage(content='你的爸爸是谁呢？', additional_kwargs={}, example=False)]

In [10]:
from langchain.schema import SystemMessage
from langchain.schema import HumanMessage
from langchain.schema import AIMessage

#直接创建消息
sy = SystemMessage(
    content = "你是一个起名大师"
)

hu = HumanMessage(
    content = "请问大师叫什么？"
)

ai = AIMessage(
    content = "我叫陈瞎子"
)

[sy, hu, ai]

[SystemMessage(content='你是一个起名大师', additional_kwargs={}),
 HumanMessage(content='请问大师叫什么？', additional_kwargs={}, example=False),
 AIMessage(content='我叫陈瞎子', additional_kwargs={}, example=False)]

自定义模板

In [14]:
import os
os.environ["OPENAI_KEY"] = "sk-Mwt9L7ljUXVAkST9dgU6itxsumV9g4A1Eykxk80QVlAMCJP5"
os.environ["OPENAI_API_BASE"] = "https://openkey.cloud/v1"

##函数大师：根据函数名称，查找函数代码，并给出中文的代码说明

from langchain.prompts import StringPromptTemplate


# 定义一个简单的函数作为示例效果
def hello_world(abc):
    print("Hello, world!")
    return abc


PROMPT = """\
你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称: {function_name}
源代码:
{source_code}
代码解释:
"""

import inspect


def get_source_code(function_name):
    #获得源代码
    return inspect.getsource(function_name)

#自定义的模板class
class CustmPrompt(StringPromptTemplate):


    def format(self, **kwargs) -> str:
        # 获得源代码
        source_code = get_source_code(kwargs["function_name"])

        # 生成提示词模板
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__, source_code=source_code
        )
        return prompt

a = CustmPrompt(input_variables=["function_name"])
pm = a.format(function_name=hello_world)

print(pm)

#和LLM连接起来
from langchain.llms import OpenAI
import os
api_base = os.getenv("OPENAI_API_BASE")
api_key = os.getenv("OPENAI_KEY")

llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key=api_key,
    openai_api_base=api_base
    )
msg = llm.predict(pm)
print(msg)

你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称: hello_world
源代码:
def hello_world(abc):
    print("Hello, world!")
    return abc

代码解释:

函数名称: hello_world
源代码:
def hello_world(abc):
    print("Hello, world!")
    return abc

代码解释:
这是一个名为hello_world的函数，它接受一个参数abc，并打印出"Hello, world!"，最后返回参数abc。它的作用是打印出"Hello, world!"这句话，并将参数abc返回。
